<a href="https://colab.research.google.com/github/asustevenma/ASU_Projects/blob/master/%5BPrototype%5D%5BMachine_Learning%5D_Marketing_Mix_Modeling(LightweightMMM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **BigQuery Magics**
BigQuery magics are used to run BigQuery SQL queries in a python environment.
These queries can also be run in the BigQuery UI

[Getting started with BigQuery - Colaboratory - Google Colab](https://colab.research.google.com/notebooks/bigquery.ipynb)

# **Set Environment**

In [ ]:
# SET BigQuery PROJECT ID #
project_id = "app-dmp-light-ga4-dev"

# Google credentials authentication libraries
from google.colab import auth
from google.colab import files
auth.authenticate_user()

# BigQuery Magics
from google.cloud import bigquery
from google.cloud import storage
from google.cloud.bigquery import magics

magics.context.project = project_id
client = bigquery.Client(project=magics.context.project)

%load_ext google.cloud.bigquery
%load_ext google.colab.data_table
bigquery.USE_LEGACY_SQL = False

In [ ]:
# Install lightweight_mmm (If you face the error that requests to restart runtime, Click the restart runtime button and re-install it)

# !pip install --upgrade pip
# !pip install lightweight_mmm

# %pip install --upgrade git+https://github.com/google/lightweight_mmm.git

## **Import Python Libraries**

In [ ]:
# Data Processing Libraries
import numpy as np
import pandas as pd
import pandas_gbq

# Modeling and Metrics
# import statsmodels.api as sm
# from statsmodels.stats.stattools import durbin_watson
# from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

# Visutalization
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.express as px

In [ ]:
# Import jax.numpy and any other library we might need
import jax.numpy as jnp
import numpyro

# Import the relevant modules of the library
from lightweight_mmm import lightweight_mmm
from lightweight_mmm import optimize_media
from lightweight_mmm import plot
from lightweight_mmm import preprocessing
from lightweight_mmm import utils

ModuleNotFoundError: No module named 'numpyro'

## **Import the Dataset from BigQuery**

In [ ]:
# Import a dataset from BigQuery table into a Pandas Dataframe

%%bigquery df_mmm_demo_raw_data

# Import a dataset from BigQuery table into a Pandas Dataframe

WITH SALES AS (
  SELECT
    ad_date,
    DATE_TRUNC(ad_date, WEEK(SUNDAY)) AS week_of_sale,
    SUM(conversion_value)*1000 AS sales_amount
  FROM `app-dmp-light-ga4-dev.bi_demo.marketing__ad_performance`
  WHERE ad_date BETWEEN DATE(2023, 1, 1) AND DATE(2023, 4, 30)
  GROUP BY 1, 2
),

CHANNEL_GOOGLE AS (
  SELECT
    DATE_SUB(ad_date, INTERVAL 1 YEAR) AS ad_date,
    DATE_TRUNC(DATE_SUB(ad_date, INTERVAL 1 YEAR), WEEK(SUNDAY)) AS week_of_sale,
    SUM(impressions) AS impressions_channel_google,
    SUM(clicks) AS clicks_channel_google,
    SUM(ad_spend) AS ad_spend_channel_google
  FROM `app-dmp-light-ga4-dev.transform_abc_mart_vpon.marketing__channel_google_ads`
  WHERE DATE_SUB(ad_date, INTERVAL 1 YEAR) BETWEEN DATE(2023, 1, 1) AND DATE(2023, 4, 30)
  GROUP BY 1, 2
),

CHANNEL_FB AS (
  SELECT
    DATE_SUB(ad_date, INTERVAL 1 YEAR) AS ad_date,
    DATE_TRUNC(DATE_SUB(ad_date, INTERVAL 1 YEAR), WEEK(SUNDAY)) AS week_of_sale,
    SUM(impressions) AS impressions_channel_fb,
    SUM(clicks) AS clicks_channel_fb,
    SUM(ad_spend) AS ad_spend_channel_fb
  FROM `app-dmp-light-ga4-dev.transform_abc_mart_vpon.marketing__channel_facebook_ads`
  WHERE DATE_SUB(ad_date, INTERVAL 1 YEAR) BETWEEN DATE(2023, 1, 1) AND DATE(2023, 4, 30)
  GROUP BY 1, 2
),

CHANNEL_IG AS (
  SELECT
    ad_date,
    DATE_TRUNC(ad_date, WEEK(SUNDAY)) AS week_of_sale,
    SUM(impressions)*5 AS impressions_channel_ig,
    SUM(clicks)*5 AS clicks_channel_ig,
    SUM(ad_spend)*5 AS ad_spend_channel_ig
  FROM `app-dmp-light-ga4-dev.bi_demo.marketing__ad_performance`
  WHERE ad_channel = "line"
    AND ad_date BETWEEN DATE(2023, 1, 1) AND DATE(2023, 4, 30)
  GROUP BY 1, 2
),

TW_HOLIDAY AS (
  SELECT
    DATE(holiday_date) AS holiday_date,
    holiday_name
  FROM `app-dmp-light-ga4-dev.data_at_bqml.holidays_by_country`
  WHERE country_code = "TW"
  GROUP BY 1, 2
)

SELECT
  SALES.ad_date,
  EXTRACT(MONTH FROM SALES.week_of_sale) AS month_of_sale,
  ROW_NUMBER() OVER (PARTITION BY EXTRACT(MONTH FROM SALES.week_of_sale) ORDER BY EXTRACT(WEEK FROM SALES.week_of_sale)) AS week_of_month,
  IF(holiday_name IS NOT NULL, 1, 0) AS is_holiday,
  impressions_channel_google,
  impressions_channel_fb,
  impressions_channel_ig,
  ad_spend_channel_google,
  ad_spend_channel_fb,
  ad_spend_channel_ig,
  sales_amount
FROM SALES
LEFT JOIN CHANNEL_GOOGLE
  ON SALES.ad_date = CHANNEL_GOOGLE.ad_date
LEFT JOIN CHANNEL_FB
  ON SALES.ad_date = CHANNEL_FB.ad_date
LEFT JOIN CHANNEL_IG
  ON SALES.ad_date = CHANNEL_IG.ad_date
LEFT JOIN TW_HOLIDAY
  ON SALES.ad_date = TW_HOLIDAY.holiday_date
ORDER BY 1, 2
;

In [ ]:
df_mmm_demo_raw_data["ad_date"] = pd.to_datetime(df_mmm_demo_raw_data["ad_date"])
df_mmm_demo_raw_data.info()

# **Exploratory Data Analysis**

In [ ]:
# plt.figure(figsize=(15,6))
# heatmap = sns.heatmap(df_mmm_demo_raw_data[["ad_spend_channel_google", "ad_spend_channel_fb", "ad_spend_channel_ig", "sales_amount"]].corr(), annot=True, cmap="Blues")

In [ ]:
# sns.pairplot(df_mmm_demo_raw_data[["ad_spend_channel_google", "ad_spend_channel_fb", "ad_spend_channel_ig", "sales_amount"]])

# **LightweightMMM**

[LightweightMMM Documentation](https://lightweight-mmm.readthedocs.io/en/latest/index.html#)

[Lightweight (Bayesian) Marketing Mix Modeling](https://github.com/google/lightweight_mmm)

Reference -> [Simple End to End Demo Script](https://github.com/takechanman1228/mmm_pydata_global_2022/blob/main/simple_end_to_end_demo_pydataglobal.ipynb)

## **Data Preprocessing**

### **Split Training and Testing Data**

In [ ]:
# Since it is time series data, we need to split training data and test data by date
split_point = pd.Timestamp("2023-04-01")

df_mmm_demo_data_train = df_mmm_demo_raw_data.loc[df_mmm_demo_raw_data["ad_date"] < split_point]
df_mmm_demo_data_test = df_mmm_demo_raw_data.loc[df_mmm_demo_raw_data["ad_date"] >= split_point]

In [ ]:
# Media data:
# Containing the metric per channel and time span (eg. impressions per time period). Media values must not contain negative values.

media_data_train = df_mmm_demo_data_train[["impressions_channel_google",
                                           "impressions_channel_fb",
                                           "impressions_channel_ig"
                                           ]].fillna(0).to_numpy().astype("float64")
media_data_test = df_mmm_demo_data_test[["impressions_channel_google",
                                         "impressions_channel_fb",
                                         "impressions_channel_ig"
                                         ]].fillna(0).to_numpy().astype("float64")

In [ ]:
# Extra Data:
# Any other features that one might want to add to the analysis.

extra_data_train = df_mmm_demo_data_train[["month_of_sale", "week_of_month", "is_holiday"]].to_numpy().astype("float64")
extra_data_test = df_mmm_demo_data_test[["month_of_sale", "week_of_month", "is_holiday"]].to_numpy().astype("float64")

In [ ]:
# Cost Data:
# The total cost per media unit per channel.

cost_data_train = df_mmm_demo_data_train[["ad_spend_channel_google",
                                          "ad_spend_channel_fb",
                                          "ad_spend_channel_ig"
                                          ]].sum().to_numpy().astype("float64")
cost_data_test = df_mmm_demo_data_test[["ad_spend_channel_google",
                                        "ad_spend_channel_fb",
                                        "ad_spend_channel_ig"
                                        ]].sum().to_numpy().astype("float64")

In [ ]:
# Target Data:
# Target KPI for the model to predict. For example, revenue amount, number of app installs.

target_data_train = df_mmm_demo_data_train["sales_amount"].fillna(0).to_numpy().astype("float64")
target_data_test = df_mmm_demo_data_test["sales_amount"].fillna(0).to_numpy().astype("float64")

### **Scale the Data**

In [ ]:
# Prepare scaler
media_data_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
extra_data_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
cost_data_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean, multiply_by=0.25) # , multiply_by=0.15
target_data_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)


# Scale the data
media_data_train_scaled = media_data_scaler.fit_transform(media_data_train)
extra_data_train_scaled = extra_data_scaler.fit_transform(extra_data_train)
cost_data_train_scaled = cost_data_scaler.fit_transform(cost_data_train)
target_data_train_scaled = target_data_scaler.fit_transform(target_data_train)

media_data_test_scaled = media_data_scaler.fit_transform(media_data_test)
extra_data_test_scaled = extra_data_scaler.fit_transform(extra_data_test)

In [ ]:
# Assign Media(Channel) Name
media_names = ["Google", "Facebook", "Instagram"]

## **Model Tuning**

The currently available models are the following:
- hill_adstock
- adstock
- carryover

In [ ]:
# Set Up Hyperparameters

SEED = 105
number_warmup=1000
number_samples=1000
adstock_models = ["adstock", "hill_adstock", "carryover"] # , "carryover"
degrees_season = [1,2,3]

list_model_name = []
list_degree = []
list_mape = []
tuning_results = {}

In [ ]:
# Hyperparameter Tuning

for model_name in adstock_models:
  for degrees in degrees_season:
    mmm_model_tune = lightweight_mmm.LightweightMMM(model_name=model_name)
    mmm_model_tune.fit(
        media=media_data_train_scaled,  # Media input data
        media_prior=cost_data_train_scaled,  # Costs of each media channel
        target=target_data_train_scaled,  # Target KPI to use
        extra_features=extra_data_train_scaled,  # Other variables to add to the model
        number_warmup=number_warmup,
        number_samples=number_samples,
        number_chains=2,  # Number of chains to sample. Default is 2
        degrees_seasonality=degrees,  # Number of degrees to use for seasonality. Default is 2
        weekday_seasonality=True,  # In case of daily data, also estimate seven weekday parameters
        seasonality_frequency=365,  # Frequency of the time period used. Default is 52 as in 52 weeks per year
        media_names=media_names,  # Names of the media channels passed
        seed=SEED
        )

    mmm_predictions_tune = mmm_model_tune.predict(
        media=media_data_test_scaled,
        extra_features=extra_data_test_scaled,
        # media_gap=0,
        target_scaler=target_data_scaler,
        seed=SEED
        )

    prediction_tune = np.mean(mmm_predictions_tune, axis=0)

    mape = mean_absolute_percentage_error(target_data_test, prediction_tune)
    # print(f"model_name={model_name} degrees={degrees} MAPE={mape} samples={prediction_tune[:3]}")

    list_model_name.append(model_name)
    list_degree.append(degrees)
    list_mape.append(mape)


tuning_results["model_name"] = list_model_name
tuning_results["degrees"] = list_degree
tuning_results["mape"] = list_mape

## **Model Training**

In [ ]:
# Preprocess all data for retrain

media_data_all = df_mmm_demo_raw_data[["impressions_channel_google",
                                       "impressions_channel_fb",
                                       "impressions_channel_ig"
                                      ]].fillna(0).to_numpy().astype("float64")

extra_data_all = df_mmm_demo_raw_data[["month_of_sale", "week_of_month", "is_holiday"]].to_numpy().astype("float64")

cost_data_all = df_mmm_demo_raw_data[["ad_spend_channel_google",
                                      "ad_spend_channel_fb",
                                      "ad_spend_channel_ig"
                                    ]].sum().to_numpy().astype("float64")

target_data_all = df_mmm_demo_raw_data["sales_amount"].fillna(0).to_numpy().astype("float64")

In [ ]:
# Prepare scaler for all the data

media_data_scaler_all = preprocessing.CustomScaler(divide_operation=jnp.mean)
extra_data_scaler_all = preprocessing.CustomScaler(divide_operation=jnp.mean)
cost_data_scaler_all = preprocessing.CustomScaler(divide_operation=jnp.mean, multiply_by=0.25) # , multiply_by=0.15
target_data_scaler_all = preprocessing.CustomScaler(divide_operation=jnp.mean)

# Scale all the data

media_data_all_scaled = media_data_scaler_all.fit_transform(media_data_all)
extra_data_all_scaled = extra_data_scaler_all.fit_transform(extra_data_all)
cost_data_all_scaled = cost_data_scaler_all.fit_transform(cost_data_all)
target_data_all_scaled = target_data_scaler_all.fit_transform(target_data_all)

In [ ]:
# Retrain the model for all data based on the results of hyperparameter tuning

min_pos = tuning_results["mape"].index(min(tuning_results["mape"]))

mmm_model = lightweight_mmm.LightweightMMM(model_name=tuning_results["model_name"][min_pos])
mmm_model.fit(
    media=media_data_all_scaled,
    media_prior=cost_data_all_scaled,
    target=target_data_all_scaled,
    extra_features=extra_data_all_scaled,
    number_warmup=number_warmup,
    number_samples=number_samples,
    number_chains=2,
    degrees_seasonality=tuning_results["degrees"][min_pos],
    weekday_seasonality=True,
    seasonality_frequency=365,
    media_names=media_names,
    seed=SEED
    )

## **Model Evaluation & Insights**

In [ ]:
mmm_model.print_summary()

In [ ]:
# Plots the ground truth, predicted value and interval for the training data.

plot.plot_model_fit(
    media_mix_model=mmm_model,
    target_scaler=target_data_scaler_all,
    interval_mid_range=0.9
    )

In [ ]:
# It estimates the media contribution percentage and ROI of each channel.
# If data was scaled prior to training then the target and costs scalers need to be passed to this function to correctly calculate media contribution percentage and ROI in the unscaled space.

media_contribution, roi_hat = mmm_model.get_posterior_metrics(
    unscaled_costs=None,
    target_scaler=target_data_scaler_all,
    cost_scaler=cost_data_scaler_all
    )

df_roi_hat = pd.DataFrame(roi_hat, columns=["Google", "Facebook", "Instagram"])
df_media_contribution = pd.DataFrame(media_contribution, columns=["Google", "Facebook", "Instagram"])

In [ ]:
# Plots a barchart of estimated media effects with their percentile interval.
# The ROI plot takes into account not only the media effect but how much it costs to get this effect.

plot.plot_bars_media_metrics(
    metric=roi_hat,
    metric_name="ROI Hat",
    channel_names=media_names
    )

In [ ]:
# Plots a barchart of estimated media effects with their percentile interval.
# Visualize the estimated contribution of each media channel

plot.plot_bars_media_metrics(
    metric=media_contribution,
    metric_name="Media Contribution Percentage",
    channel_names=media_names
    )

In [ ]:
# The media effects plot shows an estimate of the coefficients for each media channel. High numbers mean the channel influenced the revenue more.
# The x-axis is the estimated coefficient, the y-axis can be seen as how confident the model is that the x-axis value is the right value for the media effect.

plot.plot_media_channel_posteriors(
    media_mix_model=mmm_model,
    channel_names=media_names,
    quantiles=(0.05, 0.5, 0.95),
    fig_size=None
    )

In [ ]:
# Plots an area chart to visualize weekly media & baseline contribution.
# We can quickly visualize the estimated media & baseline contribution over time.

plot.plot_media_baseline_contribution_area_plot(media_mix_model=mmm_model,
                                                target_scaler=target_data_scaler_all,
                                                channel_names=media_names,
                                                fig_size=(25,15)
                                                )

In [ ]:
# Plots the response curves of each media channel based on the model.
# Visualize how each media channel behaves individually as we invest more in it

plot.plot_response_curves(
    media_mix_model=mmm_model,
    media_scaler=media_data_scaler_all,
    target_scaler=target_data_scaler_all,
    seed=SEED
    )

## **Model Prediction on out-of-sample Data**

In [ ]:
# Scale the test/out-of-sampe data if we have not done so

mmm_predictions = mmm_model.predict(
    media=media_data_test,
    extra_features=extra_data_test_scaled,
    target_scaler=target_data_scaler_all,
    seed=SEED)

prediction_array = np.mean(mmm_predictions, axis=0)

In [ ]:
df_mmm_demo_data_test["prediction"] = prediction_array.tolist()

In [ ]:
mape_test_data = mean_absolute_percentage_error(target_data_test, prediction_array)
print(f"MAPE={mape_test_data} samples={prediction_array[:3]}")

In [ ]:
# Plots the ground truth, predicted value and interval for the test data.

plot.plot_out_of_sample_model_fit(
    out_of_sample_predictions=mmm_predictions,
    out_of_sample_target=target_data_test,
    interval_mid_range=0.9
    )

## **Budget Optimization**

The optimization is meant to solve the budget allocation questions for you. First you need to provide for how long you want to optimize your budget (eg. 15 weeks).

The optimization values will be bounded by +- 20% of the max and min historic values used for training. Which means the optimization won't recommend to completely change your strategy but how to make some budget re-allocation.

Prices are the average price you would expect for the media units of each channel. If your data is already a money unit (eg. $) your prices should be an array of 1s.

In [ ]:
# Run media optimization.

estimate_budget = 800000  # The budget you want to allocate for the next n_time_periods
estimate_prices = np.array([0.1, 0.11, 0.12])  # Price per media unit per channel

In [ ]:
# Run optimization with the parameters of choice.

solution, kpi_without_optim, previous_media_allocation = optimize_media.find_optimal_budgets(
    n_time_periods=extra_data_test_scaled.shape[0],  # The number of time periods you want to simulate
    media_mix_model=mmm_model,
    extra_features=extra_data_test_scaled,
    budget=estimate_budget,
    prices=estimate_prices,
    media_scaler=media_data_scaler_all,
    target_scaler=target_data_scaler_all,
    seed=SEED
    )

If your media data is not in money unit (eg. impressions, clicks, GRPs, etc.), you would need to store the cost per values (eg. CPC) in the prices array and multiply it by solution.x to get the recommended budget allocation.

In [ ]:
# Obtain the optimal weekly allocation.
optimal_buget_allocation = estimate_prices * solution.x

# Similar renormalization to get previous budget allocation
previous_budget_allocation = estimate_prices * previous_media_allocation

In [ ]:
dict_budget_allocation = {}
dict_budget_allocation["ad_channel"] = media_names
dict_budget_allocation["previous_budget_allocation"] = previous_budget_allocation.tolist()
dict_budget_allocation["optimal_buget_allocation"] = optimal_buget_allocation.tolist()
df_budget_allocation = pd.DataFrame.from_dict(dict_budget_allocation)

In [ ]:
# Both values should be very close in order to compare KPI
estimate_budget, optimal_buget_allocation.sum()

In [ ]:
dict_pre_post_optimization = {
    "pre_post_optimization": ["pre_optimization_predicted_target", "post_optimization_predicted_target"],
    "pre_post_optimization_value": [-kpi_without_optim.item(), -solution["fun"]]
}

df_pre_post_optimization = pd.DataFrame(dict_pre_post_optimization)

In [ ]:
# Plot out pre post optimization budget allocation and predicted target variable comparison.
# The graph shows the previous budget allocation and optimized budget allocation

plot.plot_pre_post_budget_allocation_comparison(
    media_mix_model=mmm_model,
    kpi_with_optim=solution["fun"],
    kpi_without_optim=kpi_without_optim,
    optimal_buget_allocation=optimal_buget_allocation,
    previous_budget_allocation=previous_budget_allocation,
    figure_size=(10,10),
    channel_names=media_names
    )

## **Save Predictions to BigQuery Table**

In [ ]:
df_contribution = plot.create_media_baseline_contribution_df(
    media_mix_model=mmm_model,
    target_scaler=target_data_scaler_all,
    channel_names=media_names
    )

df_contribution.columns = df_contribution.columns.str.replace(" ", "_")
df_date = df_mmm_demo_raw_data[["ad_date"]].drop_duplicates().sort_values(by=["ad_date"], ignore_index=True)
df_contribution_date = pd.merge(df_contribution, df_date, left_index=True, right_index=True). #


In [ ]:
# Import the Dataframe of budget optimization into a BigQuery table

table_id = "data_at_bqml.lightweightmmm_budget_optimization"
pandas_gbq.to_gbq(dataframe=df_budget_allocation, destination_table=table_id, project_id=project_id, if_exists="replace")

In [ ]:
# Import the Dataframe of out-of-sample predictions into a BigQuery table

table_id = "data_at_bqml.lightweightmmm_prediction"
pandas_gbq.to_gbq(dataframe=df_mmm_demo_data_test, destination_table=table_id, project_id=project_id, if_exists="replace")

In [ ]:
# Import the Dataframe of contribution into a BigQuery table

table_id = "data_at_bqml.lightweightmmm_contribution"
pandas_gbq.to_gbq(dataframe=df_contribution_date, destination_table=table_id, project_id=project_id, if_exists="replace")

In [ ]:
# Import the Dataframe of pre/post budget optimization into a BigQuery table

table_id = "data_at_bqml.lightweightmmm_pre_post_optimization"
pandas_gbq.to_gbq(dataframe=df_pre_post_optimization, destination_table=table_id, project_id=project_id, if_exists="replace")

In [ ]:
# Import the Dataframe of ROI hat into a BigQuery table

table_id = "data_at_bqml.lightweightmmm_roi_hat"
pandas_gbq.to_gbq(dataframe=df_roi_hat, destination_table=table_id, project_id=project_id, if_exists="replace")

In [ ]:
# Import the Dataframe of media contribution percentage into a BigQuery table

table_id = "data_at_bqml.lightweightmmm_media_contribution_pct"
pandas_gbq.to_gbq(dataframe=df_media_contribution, destination_table=table_id, project_id=project_id, if_exists="replace")